Documentación:
https://developers.google.com/optimization/introduction/python

Importamos las librerías necesarias

In [1]:
from ortools.init.python import init
from ortools.linear_solver import pywraplp

Declaramos el solver

In [2]:
# Create the linear solver with the GLOP backend.
solver = pywraplp.Solver.CreateSolver("GLOP")
if not solver:
    print("Could not create solver GLOP")

Se crean las variables del problema (con restricciones incluidas)

In [3]:
x_var = solver.NumVar(0, 1, "x") # 0 <= x <= 1
y_var = solver.NumVar(0, 2, "y") # 0 <= y <= 2

print("Number of variables =", solver.NumVariables())

Number of variables = 2


Se definen las restricciones

In [4]:
infinity = solver.infinity()
# x + y <= 2
constraint = solver.Constraint(-infinity, 2, "ct")
constraint.SetCoefficient(x_var, 1)
constraint.SetCoefficient(y_var, 1)

print("Number of constraints =", solver.NumConstraints())

Number of constraints = 1


Definir la función objetivo

In [5]:
# f = 3 * x + y.
objective = solver.Objective()
objective.SetCoefficient(x_var, 3)
objective.SetCoefficient(y_var, 1)
objective.SetMaximization()

Llamar al solver e imprimir los resultados

In [6]:
print(f"Solving with {solver.SolverVersion()}")
result_status = solver.Solve()
print(f"Status: {result_status}")
if result_status != pywraplp.Solver.OPTIMAL:
    print("The problem does not have an optimal solution!")
    if result_status == pywraplp.Solver.FEASIBLE:
        print("A potentially suboptimal solution was found")
    else:
        print("The solver could not solve the problem.")

print("Solution:")
print("Objective value =", objective.Value())
print("x =", x_var.solution_value())
print("y =", y_var.solution_value())

Solving with Glop solver v9.11.4210
Status: 0
Solution:
Objective value = 4.0
x = 1.0
y = 1.0


### Problemas de asignación

Creamos los datos:

In [7]:
costs = [
    [90, 80, 75, 70],
    [35, 85, 55, 65],
    [125, 95, 90, 95],
    [45, 110, 95, 115],
    [50, 100, 90, 100],
]
num_workers = len(costs)
num_tasks = len(costs[0])

Se declara el solver

In [8]:
# Create the mip solver with the SCIP backend.
solver = pywraplp.Solver.CreateSolver("SCIP")

Se crean variables de números enteros binarios donde la variable x[i,j] indica si el trabajador i se asignará a la tarea j

In [9]:
x = {}
for i in range(num_workers):
    for j in range(num_tasks):
        x[i, j] = solver.IntVar(0, 1, "")

Se crean las restricciones. Por la naturaleza del problema, las básicas son que cada trabajador se asigna como mucho a una tarea y que cada tarea tenga asignado como máximo a un trabajador.

In [10]:
# 1 tarea por trabajador como máximo
for i in range(num_workers):
    solver.Add(solver.Sum([x[i, j] for j in range(num_tasks)]) <= 1)

# 1 trabajador por tarea como exactamente
for j in range(num_tasks):
    solver.Add(solver.Sum([x[i, j] for i in range(num_workers)]) == 1)

Se crea la función objetivo. En este caso el coste.

In [11]:
objective_terms = []
for i in range(num_workers):
    for j in range(num_tasks):
        objective_terms.append(costs[i][j] * x[i, j])
solver.Minimize(solver.Sum(objective_terms))

Se llama al solver

In [12]:
print(f"Solving with {solver.SolverVersion()}")
status = solver.Solve()

Solving with SCIP 9.0.0 [LP solver: Glop 9.11]


Se imprime la solución

In [13]:
if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
    print(f"Total cost = {solver.Objective().Value()}\n")
    for i in range(num_workers):
        for j in range(num_tasks):
            # Test if x[i,j] is 1 (with tolerance for floating point arithmetic).
            if x[i, j].solution_value() > 0.5:
                print(f"Worker {i} assigned to task {j}." + f" Cost: {costs[i][j]}")
else:
    print("No solution found.")

Total cost = 265.0

Worker 0 assigned to task 3. Cost: 70
Worker 1 assigned to task 2. Cost: 55
Worker 2 assigned to task 1. Cost: 95
Worker 3 assigned to task 0. Cost: 45
